eutai file ma name ra data lyako

In [1]:
import pandas as pd

# Load the datasets
filename_df = pd.read_csv('filename.csv')  # Replace with the actual path to 'filename.csv'
phoneme_df = pd.read_csv('padded_phoneme_sequences.csv')  # Replace with the actual path to 'padded_phoneme_sequences.csv'

# Merge the datasets by adding filenames to the padded phoneme sequences
merged_df = pd.DataFrame({
    'filename': filename_df['filename'],
    'padded_phonemes': phoneme_df['padded_phonemes']
})

# Save the merged DataFrame to a new CSV file
output_path = 'merged_filename_phonemes.csv'  # Update the path accordingly
merged_df.to_csv(output_path, index=False)

print(f"Merged file saved at: {output_path}")


Merged file saved at: merged_filename_phonemes.csv


In [2]:
import pandas as pd

# Load the datasets
filename_df = pd.read_csv('filename.csv')  # Replace with the actual path to 'filename.csv'
phoneme_df = pd.read_csv('padded_phoneme_sequences.csv')  # Replace with the actual path to 'padded_phoneme_sequences.csv'

# Assuming the 'padded_phonemes' column contains lists or sequences, compute their lengths
phoneme_df['length'] = phoneme_df['padded_phonemes'].apply(lambda x: len(eval(x)) if isinstance(x, str) else len(x))

# Merge the datasets by adding filenames to the padded phoneme sequences
merged_df = pd.DataFrame({
    'filename': filename_df['filename'],
    'padded_phonemes': phoneme_df['padded_phonemes'],
    'length': phoneme_df['length']
})

# Save the merged DataFrame to a new CSV file
output_path = 'merged_filename_phonemes_with_lengths.csv'  # Update the path accordingly
merged_df.to_csv(output_path, index=False)

print(f"Merged file with lengths saved at: {output_path}")


Merged file with lengths saved at: merged_filename_phonemes_with_lengths.csv


In [3]:
import pandas as pd

# File paths
phoneme_file = r'D:\7semproject\TTSS\merged_filename_phonemes_with_lengths.csv'
text_file = r'D:\7semproject\TTSS\raw_text.csv'
output_file = r'D:\7semproject\TTSS\combined_data.csv'

# Load CSV files
phoneme_df = pd.read_csv(phoneme_file)
text_df = pd.read_csv(text_file)

# Merge on the 'filename' column
combined_df = pd.merge(phoneme_df, text_df, on='filename', how='inner')

# Rename columns to match the required format
combined_df = combined_df[['filename', 'transcription', 'padded_phonemes', 'length']]

# Save to new CSV file
combined_df.to_csv(output_file, index=False)

print("Merged data saved to", output_file)


Merged data saved to D:\7semproject\TTSS\combined_data.csv


In [7]:
import pandas as pd
import os

# Define paths
raw_audio_dir = r'D:\7semproject\LjSpeech\wavs'
processed_audio_dir = r'D:\7semproject\TTSS\log_scaled_mel_spectrograms'
mel_lengths_path = r'D:\7semproject\TTSS\mel_lengths.csv'

# Read mel lengths CSV
mel_lengths_df = pd.read_csv(mel_lengths_path)

# Ensure 'filename' column exists in the CSV
if 'filename' in mel_lengths_df.columns:
    # Remove any row where 'filename' is 'max_mel_length' to avoid extra entry
    mel_lengths_df = mel_lengths_df[mel_lengths_df['filename'] != 'max_mel_length']

    # Remove '.npy' extension from the filename column if present
    mel_lengths_df['filename'] = mel_lengths_df['filename'].str.replace('.npy', '', regex=False)

    # Add columns for full paths to raw and processed audio files
    mel_lengths_df['raw_audio_path'] = mel_lengths_df['filename'].apply(lambda x: os.path.join(raw_audio_dir, f"{x}.wav"))
    mel_lengths_df['processed_audio_path'] = mel_lengths_df['filename'].apply(lambda x: os.path.join(processed_audio_dir, f"{x}.npy"))

    # Reorder columns for clarity
    mel_lengths_df = mel_lengths_df[['filename', 'raw_audio_path', 'processed_audio_path', 'mel_length']]

    # Save the combined data to a new CSV file
    output_csv = r'D:\7semproject\TTSS\combined_audio_metadata.csv'
    mel_lengths_df.to_csv(output_csv, index=False)

    print(f"Combined audio metadata saved to {output_csv}")
else:
    print("Error: 'filename' column not found in the CSV file.")


Combined audio metadata saved to D:\7semproject\TTSS\combined_audio_metadata.csv


In [9]:
import pandas as pd

# Load the data files
combined_data = pd.read_csv('combined_data.csv')
audio_metadata = pd.read_csv('combined_audio_metadata.csv')

# Check for common filenames
common_filenames = set(combined_data['filename']).intersection(set(audio_metadata['filename']))

if not common_filenames:
    print("No common filenames found between the two files.")
    print(f"Unique filenames in combined_data: {combined_data['filename'].nunique()}")
    print(f"Unique filenames in audio_metadata: {audio_metadata['filename'].nunique()}")
    print("Sample filenames from combined_data:", combined_data['filename'].head().tolist())
    print("Sample filenames from audio_metadata:", audio_metadata['filename'].head().tolist())
else:
    print(f"Found {len(common_filenames)} common filenames. Proceeding with merge.")

    # Merge the dataframes based on the common filenames
    merged_df = pd.merge(combined_data, audio_metadata, on='filename', how='inner')
    print(f"Merged DataFrame has {merged_df.shape[0]} rows.")

    # Define the output path for the merged dataset
    output_csv = 'merged_dataset.csv'
    
    # Save the merged dataframe to a CSV file
    merged_df.to_csv(output_csv, index=False)
    print(f"Merged dataset saved to {output_csv}")


Found 13100 common filenames. Proceeding with merge.
Merged DataFrame has 13100 rows.
Merged dataset saved to merged_dataset.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset from a .txt file
input_file = r'D:\Project\list.txt'  # Replace with the path to your .txt file
df = pd.read_csv(input_file, sep='|', names=['raw_audio_path', 'processed_audio_path', 'transcription', 'mel_length'])

# Split the dataset into training, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

# Verify the splits
print(f"Training set: {len(train_df)} samples")
print(f"Validation set: {len(val_df)} samples")
print(f"Test set: {len(test_df)} samples")

# Function to create metadata files
def create_metadata_file(df, output_path):
    with open(output_path, 'w') as f:
        for _, row in df.iterrows():
            audio_path = row['raw_audio_path']
            mel_path = row['processed_audio_path']
            text = row['transcription']
            mel_length = row['mel_length']
            f.write(f"{audio_path}|{mel_path}|{text}|{mel_length}\n")

# Paths to save the metadata files
train_metadata_path = 'train.txt'
val_metadata_path = 'val.txt'
test_metadata_path = 'test.txt'

# Create metadata files
create_metadata_file(train_df, train_metadata_path)
create_metadata_file(val_df, val_metadata_path)
create_metadata_file(test_df, test_metadata_path)

print(f"Metadata files created: {train_metadata_path}, {val_metadata_path}, {test_metadata_path}")


Training set: 10611 samples
Validation set: 1179 samples
Test set: 1310 samples
Metadata files created: train.txt, val.txt, test.txt


In [3]:
import pandas as pd

# Load the list.txt and normalized_text.csv files
list_df = pd.read_csv(r'D:\Project\list.txt', sep='|', header=None, names=['path', 'transcription'])
normalized_df = pd.read_csv(r'D:\Project\normalized_text.csv')

# Remove the transcription column from list_df
list_df = list_df.drop(columns=['transcription'])

# Add the normalized_transcription column from normalized_df as the new transcription column
list_df['transcription'] = normalized_df['normalized_transcription']

# Save the updated DataFrame back to a .txt file with pipe-separated values
list_df.to_csv(r'D:\Project\updated_list_with_normalized_transcriptions.txt', sep='|', header=False, index=False)


In [ ]:
import pandas as pd

# Load the data files
combined_data = pd.read_csv('combined_data.csv')
audio_metadata = pd.read_csv('combined_audio_metadata.csv')

# Check for common filenames
common_filenames = set(combined_data['filename']).intersection(set(audio_metadata['filename']))

if not common_filenames:
    print("No common filenames found between the two files.")
    print(f"Unique filenames in combined_data: {combined_data['filename'].nunique()}")
    print(f"Unique filenames in audio_metadata: {audio_metadata['filename'].nunique()}")
    print("Sample filenames from combined_data:", combined_data['filename'].head().tolist())
    print("Sample filenames from audio_metadata:", audio_metadata['filename'].head().tolist())
else:
    print(f"Found {len(common_filenames)} common filenames. Proceeding with merge.")

    # Merge and save merged data if common filenames exist
    merged_df = pd.merge(combined_data, audio_metadata, on='filename', how='inner')
    print(f"Merged DataFrame has {merged_df.shape[0]} rows.")

    # Proceed with data split only if merged_df is non-empty
    if not merged_df.empty:
        from sklearn.model_selection import train_test_split

        # Split into train, validation, and test sets
        train_val, test = train_test_split(merged_df, test_size=0.1, random_state=42)
        train, val = train_test_split(train_val, test_size=0.1, random_state=42)

        # Save the splits
        train.to_csv('train_data.csv', index=False)
        val.to_csv('val_data.csv', index=False)
        test.to_csv('test_data.csv', index=False)

        print("Data split into train, validation, and test sets.")
    else:
        print("The merged DataFrame is empty after attempting to merge.")


cleaning

In [26]:
import csv

def check_and_correct_csv(file_path, output_path, expected_columns=8):
    """
    Check and correct the CSV file at the given path.
    
    Args:
        file_path (str): Path to the input CSV file.
        output_path (str): Path to the output CSV file.
        expected_columns (int): The expected number of columns in the CSV.
    """
    corrected_lines = []
    problematic_lines = []

    with open(file_path, 'r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)

        for line_number, row in enumerate(reader, start=1):
            # Check if the row has the expected number of columns
            if len(row) != expected_columns:
                if len(row) > expected_columns:
                    # Assume the last column contains extra data
                    corrected_row = row[:expected_columns - 1] + [' '.join(row[expected_columns - 1:])]
                else:
                    # Attempt to fill missing values for a row with too few columns
                    corrected_row = row + [''] * (expected_columns - len(row))  # Fill with empty strings
                
                problematic_lines.append((line_number, row, "Incorrect number of columns, corrected."))
                corrected_lines.append([f'"{item}"' for item in corrected_row])  # Add quotes
                continue  # Skip further processing for this row

            # Escape unescaped quotes in each element
            try:
                corrected_row = [f'"{item.replace("\"", "\"\"")}"' for item in row]
                corrected_lines.append(corrected_row)  # Append the corrected row

            except Exception as e:
                # Log the problematic line and the reason for the failure
                problematic_lines.append((line_number, row, str(e)))

    # Write the corrected lines to a new CSV file
    with open(output_path, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(corrected_lines)

    # Output problematic lines for review
    if problematic_lines:
        print("Problematic lines found:")
        for line_number, row, reason in problematic_lines:
            print(f"Line {line_number}: {row} - Reason: {reason}")

# Example usage
check_and_correct_csv(
    'D:\\7semproject\\TTSS\\text_to_speech_tacotron2\\tacotron2\\filelists\\train_data.csv',
    'D:\\7semproject\\TTSS\\text_to_speech_tacotron2\\tacotron2\\filelists\\train_data_corrected.csv'
)


In [1]:
import os

def convert_to_utf8(input_file, output_file, source_encoding='ISO-8859-1'):
    try:
        # Open the input file with the specified source encoding
        with open(input_file, 'r', encoding=source_encoding) as infile:
            # Read the content
            content = infile.read()
        
        # Write the content to the output file in UTF-8 encoding
        with open(output_file, 'w', encoding='utf-8') as outfile:
            outfile.write(content)
        
        print(f"Successfully converted '{input_file}' to UTF-8 and saved as '{output_file}'")
    
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_file_path = r'D:\Project\list.txt'  # Replace with your input file path
output_file_path = r'D:\Project\clist.txt'  # Replace with your desired output file path

# Specify the source encoding if it's different from ISO-8859-1
convert_to_utf8(input_file_path, output_file_path, source_encoding='ISO-8859-1')


Successfully converted 'D:\Project\list.txt' to UTF-8 and saved as 'D:\Project\clist.txt'
